# Scrap/Utility Notebook for Flask App

First we are loading models from GCP Vertex AI so we can copy them in the Dockerfile and hopefully decrease latency.

In [1]:
import pickle
import json
import pandas_gbq
from sklearn.metrics.pairwise import cosine_similarity
import os
import ast
from google.cloud import storage
import numpy as np

In [2]:
gcp_config_file = 'flavourquasar-gcp-key.json'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = gcp_config_file

In [3]:
def load_artifact_from_gcs(artifact_path, bucket_name='macro_predictor'):
    """
    Loads an artifact from a pickle file stored in Google Cloud Storage.
    Args:
        bucket_name (str): Name of the GCS bucket.
        artifact_path (str): Path within the bucket where the artifact is stored.
    Returns:
        object: Loaded pickle file.
    """
    # Initialize a client and bucket
    client = storage.Client()
    bucket = client.bucket(bucket_name)

    # Get the blob (pickle file) from GCS
    blob = bucket.blob(artifact_path)

    # Download the pickle file contents as bytes
    pickle_bytes = blob.download_as_string()

    # Load the dictionary from the downloaded pickle file contents
    loaded_artifact = pickle.loads(pickle_bytes)

    return loaded_artifact

In [4]:
svd_fitted = load_artifact_from_gcs('macro_data_processing/svd_fitted.pkl')
tfidf_fitted = load_artifact_from_gcs('macro_data_processing/tfidf_fitted.pkl')

XGBoost_fat_model = load_artifact_from_gcs('training/XGBoost_fat_model.pkl')
XGBoost_carbs_model = load_artifact_from_gcs('training/XGBoost_carbs_model.pkl')
XGBoost_protein_model = load_artifact_from_gcs('training/XGBoost_protein_model.pkl')

In [7]:
type(XGBoost_fat_model)

xgboost.sklearn.XGBRegressor

In [11]:
def save_model(model, file_name, folder='models/'):
    file_path = folder + file_name
    with open(file_path, "wb") as f:
        pickle.dump(model, f)

def load_model(model_file_name, folder='models/'):
    file_path = folder + model_file_name
    with open(file_path, "rb") as f:
        model = pickle.load(f)
    return model

In [15]:
save_model(svd_fitted, 'svd_fitted.pkl')
save_model(tfidf_fitted, 'tfidf_fitted.pkl')

save_model(XGBoost_fat_model, 'XGBoost_fat_model.pkl')
save_model(XGBoost_carbs_model, 'XGBoost_carbs_model.pkl')
save_model(XGBoost_protein_model, 'XGBoost_protein_model.pkl')